___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://bank.gov.ua/ua/open-data/api-dev



1. Підключитися до API.


In [192]:
import requests
import json
import datetime
import pprint
import math
import pickle

pp = pprint.PrettyPrinter(sort_dicts=True)

def get_exchange_rate_for_period(start, end, currency = 'usd'):
  return requests.get(f'https://bank.gov.ua/NBU_Exchange/exchange_site?start={start}&end={end}&valcode={currency}&sort=exchangedate&order=asc&json')

def get_average(data):
  result = sum(row['rate_per_unit'] for row in data) / len(data)
  return round(result, 2)

2. Отримати курс долара за останній рік.

In [152]:
date_format = '%Y%m%d'
date_formatted = '%Y-%m-%d'
current_date = datetime.datetime.now().strftime(date_format)

year_ago = datetime.datetime.now() - datetime.timedelta(days=365)
year_ago_formatted = year_ago.strftime(date_format)

usd_data = get_exchange_rate_for_period(year_ago_formatted, current_date).json()
usd_price_for_year = get_average(usd_data)

# print(json.dumps(usd_data, indent=4))
print(f'Usd average for the priod: {year_ago.strftime(date_formatted)} - {datetime.datetime.now().strftime(date_formatted)} is {usd_price_for_year}')

Usd average for the priod: 2022-03-06 - 2023-03-06 is 33.83


3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [205]:
months_dict = {}
usd_average_months = {}
period = 12

def get_previous_month_date(today):
  first = today.replace(day=1)
  last_month = first - datetime.timedelta(days=1)
  return last_month

current = datetime.datetime.now()
previous = ''

for i in range(period):
  previous = get_previous_month_date(current)
  previous_start = previous.replace(day=1).strftime(date_format)
  previous_end = previous.strftime(date_format)
  start_format = previous.replace(day=1).strftime(date_formatted)
  end_format = previous.strftime(date_formatted)

  months_dict[f'{start_format} - {end_format}'] = get_exchange_rate_for_period(f'{previous_start}', f'{previous_end}').json()
  
  current = previous

average = 0

for key, value in months_dict.items():
  usd_average_months[key] = get_average(value)
  average += get_average(value)

usd_average_price = average / len(months_dict)

average_prices_sum = 0

for key, value in usd_average_months.items():
  average_prices_sum += (value - usd_average_price) ** 2

deviation = math.sqrt(average_prices_sum / len(usd_average_months))

usd_average_months.update({'deviation': round(deviation, 2)})


4. Дану інформацію записати у файл за допомогою pickle.

In [206]:
with open('./usd_price_info.pickle', 'wb') as handle:
  pickle.dump(usd_average_months, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('./usd_price_info.pickle', 'rb') as handle:
  pickle_data = pickle.load(handle)

pp.pprint(pickle_data)

{'2022-03-01 - 2022-03-31': 29.25,
 '2022-04-01 - 2022-04-30': 29.25,
 '2022-05-01 - 2022-05-31': 29.25,
 '2022-06-01 - 2022-06-30': 29.25,
 '2022-07-01 - 2022-07-31': 31.85,
 '2022-08-01 - 2022-08-31': 36.57,
 '2022-09-01 - 2022-09-30': 36.57,
 '2022-10-01 - 2022-10-31': 36.57,
 '2022-11-01 - 2022-11-30': 36.57,
 '2022-12-01 - 2022-12-31': 36.57,
 '2023-01-01 - 2023-01-31': 36.57,
 '2023-02-01 - 2023-02-28': 36.57,
 'deviation': 3.42}


### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

2. Виведіть перелік всіх таблиць.

3. Список всіх cust_id з таблиці Customers table.

4. Всю таблицю Customers table

5. Список клієнтів (cust_names) відсортованих від Z до A

6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

8. Товар (з таблиці Products), ціна якого становить 9.49

9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

10. Кількість товару, що було продано

11. Кількість найменувань товару, ціна якого більша за 4

12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди
